In [1]:
pip install pyautogui

     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ------------ ------------------------- 20.5/61.2 kB 682.7 kB/s eta 0:00:01
     ------------ ------------------------- 20.5/61.2 kB 682.7 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/61.2 kB 281.8 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/61.2 kB 281.8 kB/s eta 0:00:01
     ------------------------------- ------ 51.2/61.2 kB 262.6 kB/s eta 0:00:01
     -------------------------------------- 61.2/61.2 kB 272.7 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requir

In [2]:
from selenium import webdriver
driver = webdriver.Chrome()

In [8]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium.webdriver.common.by import By
from tqdm import tqdm

# 여러 개의 키워드 리스트
keywords = ["아침", "점심", "저녁", "오전", "오후", "밤"]

# Chrome WebDriver 설정
driver = webdriver.Chrome()

# 수집된 데이터를 저장할 리스트
dataframes = []

for keyword in keywords:
    # 해당 키워드의 데이터프레임 초기화
    keyword_data = []

    # 이전 페이지 소스 초기화
    prev_page_source = None

    # 1페이지부터 100페이지까지 크롤링
    for pageNum in tqdm(range(1, 200), desc=f"{keyword} 검색 진행 중"):
        # 네이버 사전 페이지로 이동
        driver.get(f"https://ko.dict.naver.com/#/search?range=example&page={pageNum}&query={keyword}&autoConvert=")

        # 페이지 로딩을 기다림 (3초 기다리고 조절 가능)
        time.sleep(3)

        # 현재 페이지의 소스 코드
        current_page_source = driver.page_source

        # 현재 페이지와 이전 페이지의 소스 코드가 같다면 마지막 페이지까지 크롤링한 것으로 간주
        if prev_page_source and current_page_source == prev_page_source:
            break

        # 이전 페이지 소스 업데이트
        prev_page_source = current_page_source

        # 검색 결과를 파싱
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        examples = soup.select('.origin[lang="ko"] .text')

        for example in examples:
            title = example.text.strip()
            keyword_data.append({'Title': title})

        # 다음 페이지로 이동
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "a.next")
            next_button.click()
        except NoSuchElementException:
            break

    # 수집된 데이터를 데이터프레임으로 변환하여 리스트에 추가
    keyword_df = pd.DataFrame(keyword_data)
    dataframes.append(keyword_df)

# WebDriver 종료
driver.quit()

# 키워드별로 수집된 데이터프레임을 합치기
result_df = pd.concat(dataframes, keys=keywords, names=['Keyword'])

# 결과 데이터프레임 출력
print(result_df)


밤 검색 진행 중:  67%|██████▋   | 133/199 [27:56<13:51, 12.61s/it]


                                                          Title
Keyword                                                        
아침      0                                    그는 아침 일찍 일어나는 편이다.
        1                                    그는 아침을 거른 채 출근하였다.
        2                                 아침을 설치고 왔더니 참말로 시장하오.
        3              모처럼 휴일을 맞은 젊은이들은 이른 아침부터 극장 앞에 줄을 서 있었다.
        4         아치골댁은…부뚜막에 걸터앉아 아침 겸 점심인 고구마 밥을 허기지게 먹기 시작했다.
...                                                         ...
밤       2005  귀신 이야기 하나 하지. 먼 디를 갖다 오니깨. 밤에 내려온깨, 하, 요 산에 공동...
        2006  첫날밤에 인자 신랑 신부가 자는디, 자다가 본께 그때는 오좀이 마란 것이 아니라 똥...
        2007  급한 것 몇가지만 빨아, 짜 널고는 들어와 한축을 하고 앓기 시작하였다. 몸이 불덩...
        2008  그 이듬해 그냥 막 비가 퍼붓어서 함물이 진게 그냥 밤자고 인난 게 그냥 모래가 확...
        2009  생과일 놓구, 밤, 대추 놓구, 고 뒤루 서쪽에는 포를 놓구, 동쪽 끝에는 단밥을 ...

[9237 rows x 1 columns]


In [10]:
result_df.tail(20)

Title
Keyword                                                        
밤       1990    인제 판서하는 사람한테 가닝개 그 밤새워서 이야기하고 그렇다고 하닝개, “그래야!”고
        1991  첫날밤에 인자 신랑 신부가 자는디, 자다가 본께 그때는 오좀이 마란 것이 아니라 똥...
        1992  지집애가 까탈시럽기는. 첫날밤부터 신랑한티 신바람이나 시키고. 내가 다 민망혀 죽겄구만.
        1993  그 아부지가 밤에 꿈을 꿍게, 달이, 아니 해가 훤허니 뜨더니 문구녁으로 후르르 들...
        1994  그래 거기를 갔었는데 인제 오기를 기다리는 데 밤이 야식한게, “아무것이 아무것이....
        1995  아이고, 며느리 못났다고 했더니 요새 밤을 궈다 주고, 쪄다 주고, 밥도 와서 많이...
        1996  모르긴 몰라도, 밤늦은 시각으로 미루어 아마 초저녁 무럽부터 벌이기 시작한 축사 적...
        1997  무덤산에 고사리는/밤이슬 맞고 히들어졌네/우리 집에 우리 올키/잠이 와서 히들어졌네...
        1998  그 참 담뱃대허고 담배허고 화릿불을 밤새드락 안꺼지게 갖다놓고, 안 꺼질만헌 정도로...
        1999  밥은 천상(불가피하게) 참 거그서 인자 밤에 인자 그럭을 다 치고 인자 밤에 오먼 ...
        2000  아 밤이먼은 어서(어디서) 날러 들오는가 남진일색이 그옷 우아래 쌔뜩허게 입고, 그...
        2001  죽은 애비 뒷산마루 묏동에다 파묻어 내비리고는, 자식 새끼도 팽개치고 밤도망 가 부...
        2002  텅 빈 동헌만 지키고 앉았을 터무니가 없는 것이었다. 제일에 밤이면 귀신 우는 소리...
        2003  다시 한번 얘기해 두겠는데, 밤나들인 여기서 아주 위험해. 그 짐승허구 맞닥뜨리는 ...
        2004  저 산 하나 넘어가야 있는디, 섣달 그믐 날 저녁을 캄캄한 밤을 산을 넘어가는디, ...
        2005  귀신 이야기 하나 하지. 먼 디를 갖다 오니깨. 밤에 내려온깨, 하, 요 산에 공동...
        2006  첫날밤에 인자 신랑 신부가 자는디, 자다가 본께 그때는 오좀이 마란 것이 아니라 똥...
        2007  급한 것 몇가지만 빨아, 짜 널고는 들어와 한축을 하고 앓기 시작하였다. 몸이 불덩...
        2008  그 이듬해 그냥 막 비가 퍼붓어서 함물이 진게 그냥 밤자고 인난 게 그냥 모래가 확...
        2009  생과일 놓구, 밤, 대추 놓구, 고 뒤루 서쪽에는 포를 놓구, 동쪽 끝에는 단밥을 ...

In [11]:
result_df.to_csv('keyword_dict.csv', index=False)
